## Data Mining of Solar Flares

The Data provided is from the Reuven Ramaty High Energy Solar Spectroscopic Imager (RHESSI, originally High Energy Solar Spectroscopic Imager or HESSI). It is a NASA solar flare observatory. <br>
Description and analysis of the data is in the report, here is manipulation of data in various forms filtering,cleaning, and transforming the data as appropriate such that it can be used to produce optimal classification for the flare bassed on its energy oputput, in the form of MLP and RF classifiers.


In [67]:
# Imports
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sklearn
import scikitplot as skplt

#from datetime import datetime

from sklearn.model_selection import train_test_split
# Import Model Classifiers.
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
# Import evaluation libaries.
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

# Data preperation
### Data ingestion.

In [70]:
# Function for converting string to a datetime, specifying the format.
# Datetime Formatting Codes: http://bit.ly/python-dt-fmt
# Working with Dates and Time Series Data: https://youtu.be/UFuo7EHI8zc?t=641
# Pandas can recognise the datetime format of this rhessi data, so it doesn't need this 'date_parser' method.
# But doing this in case I need it for other data.
d_parser = lambda x: datetime.strptime(x, '%Y-%m-%d')

In [71]:
df_rhessi_orig = pd.read_csv('../Data Sets/V1 Solar Flares from RHESSI Mission/hessi.solar.flare.2002to2016.csv'
, parse_dates=['start.date'], date_parser=d_parser)
# Read the original data in. 
df_rhessi = df_rhessi_orig.copy() # Create a copy to work with.
print(type(df_rhessi)) # Check it's in the expected DataFrame format.

<class 'pandas.core.frame.DataFrame'>


### Initial inspection of the data.

In [ ]:
df_rhessi.shape # Rows and Columns.

In [ ]:
df_rhessi.head(10) # Making sure the data has been injested correctly.

In [ ]:
# If you want to parse to date time after reading in the data.
#df_rhessi['start.date'] = pd.to_datetime(df_rhessi['start.date'], format='%Y-%m-%d')

In [72]:
df_rhessi['start.date'].dt.day_name()

0           Tuesday
1           Tuesday
2         Wednesday
3         Wednesday
4         Wednesday
            ...    
113937       Monday
113938       Monday
113939      Tuesday
113940      Tuesday
113941      Tuesday
Name: start.date, Length: 113942, dtype: object


### Look at the data types of each column/attribute.

In [ ]:
# They do not match the briefe, this will need to be rectified later. 
# IPSI and Contra should be of type int.
df_rhessi.dtypes

### Inspect the Unique values of the categorical (dtype object), attributes.

In [ ]:
# Create a new df_uniques and read in the columns unique values.
df_uniques = pd.DataFrame(columns= ['energykev', 'flag1', 'flag2', 'flag3', 'flag4', 'flag5'])
df_uniques.energykev = pd.Series(df_rhessi['energy.kev'].unique())
df_uniques.flag1 = pd.Series(df_rhessi["flag.1"].unique())
df_uniques.flag2 = pd.Series(df_rhessi['flag.2'].unique())
df_uniques.flag3 = pd.Series(df_rhessi['flag.3'].unique())
df_uniques.flag4 = pd.Series(df_rhessi['flag.4'].unique())
df_uniques.flag5 = pd.Series(df_rhessi['flag.5'].unique())
df_uniques

## Cleaning

First we will look for nulls and drop them for now, later revisions we will do some imputing. <br>

Fortunately the data from Kaggl is clean!! 

### Drop Null entries.

In [ ]:
print(df_rhessi.isnull().sum()) # Show nulls.

### Fixing the data types.

I was able to parse the start.date column to type datetime as I read the .csv file in. <br>

But for the time values, I would like them as just datetime.time.

In [ ]:
# From our initial inspection, we know that some of our columns are of the wrong data type.
# We want the date time to be of type date time.
df_rhessi.dtypes

In [ ]:
# https://medium.com/@vincentteyssier/optimizing-the-size-of-a-pandas-dataframe-for-low-memory-environment-5f07db3d72e
# I found that using a smaller int subtype means pandas requires less memory.
# Consuming less memory is always better!
df_rhessi['start.time'] = pd.to_datetime.time(df_rhessi['start.time'])

In [ ]:
# Because I know the values for IPSI and Contra range from 0 to 101, it makes sense to use int8, only consuming 1 byte of memory.
# To be safe I will cast to int and then downcast safely.
df_cvd_cleaned.IPSI = df_cvd_cleaned.IPSI.astype(int)
df_cvd_cleaned.IPSI = pd.to_numeric(df_cvd_cleaned.IPSI, downcast=('unsigned'))

In [ ]:
# Error when casting Contra of type Object to int, there is an empty entry.
# Find it here.  
s_empty_contra = df_cvd_cleaned[df_cvd_cleaned.Contra == " "]
s_empty_contra

In [ ]:
# Drop empty Contra here.
df_cvd_cleaned = df_cvd_cleaned.drop(index=642)

In [ ]:
# Cast Contra from object, to string, strip leading and trailing white spaces, cast to int, then downcast.
df_cvd_cleaned.Contra = df_cvd_cleaned.Contra.astype(str).str.strip().astype(int)
df_cvd_cleaned.Contra = pd.to_numeric(df_cvd_cleaned.Contra, downcast=('unsigned'))

In [ ]:
# Convert objects to nominal category, we don't care about order. Again this saves memory. 
df_cvd_cleaned.Indication = df_cvd_cleaned.Indication.astype('category')
df_cvd_cleaned.Diabetes = df_cvd_cleaned.Diabetes.astype('category')
df_cvd_cleaned.IHD = df_cvd_cleaned.IHD.astype('category')
df_cvd_cleaned.Hypertension = df_cvd_cleaned.Hypertension.astype('category')
df_cvd_cleaned.Arrhythmia = df_cvd_cleaned.Arrhythmia.astype('category')
df_cvd_cleaned.History = df_cvd_cleaned.History.astype('category')
df_cvd_cleaned.label = df_cvd_cleaned.label.astype('category')

In [ ]:
# Now are data types are more representitve of the data.
df_cvd_cleaned.dtypes

## Data visualisation.

In [ ]:
# I have created my own pallete to clearly convay risk in red and no risk in blue.
NoRisk_Risk = ["#2F70F2", "#D83A37"]
sns.set_palette(NoRisk_Risk)
sns.palplot(sns.color_palette())

### Count plots, stacked with label.

In [ ]:
plt.title("Count of Risk/NoRisk")
sns.countplot(data=df_cvd_cleaned, x='label')

In [ ]:
# plt.title("Count of Diabtese")
df_plot = df_cvd_cleaned.groupby(['label', 'Diabetes']).size().reset_index().pivot(columns='label', index='Diabetes', values=0)
df_plot.plot(kind='bar', stacked=True)
#sns.countplot(data=df_cvd_cleaned, x='Diabetes')

In [ ]:
# plt.title("Count of IHD")
df_plot = df_cvd_cleaned.groupby(['label', 'IHD']).size().reset_index().pivot(columns='label', index='IHD', values=0)
df_plot.plot(kind='bar', stacked=True)

In [ ]:
#plt.title("Count of Hypertension")
df_plot = df_cvd_cleaned.groupby(['label', 'Hypertension']).size().reset_index().pivot(columns='label', index='Hypertension', values=0)
df_plot.plot(kind='bar', stacked=True)

In [ ]:
# plt.title("Count of Arrhythmia")
df_plot = df_cvd_cleaned.groupby(['label', 'Arrhythmia']).size().reset_index().pivot(columns='label', index='Arrhythmia', values=0)
df_plot.plot(kind='bar', stacked=True)

In [ ]:
# plt.title("Count of History")
df_plot = df_cvd_cleaned.groupby(['label', 'History']).size().reset_index().pivot(columns='label', index='History', values=0)
df_plot.plot(kind='bar', stacked=True)

In [ ]:
# plt.title("Count of Indication")
df_plot = df_cvd_cleaned.groupby(['label', 'Indication']).size().reset_index().pivot(columns='label', index='Indication', values=0)
df_plot.plot(kind='bar', stacked=True)

### Looking at IPSI and Contra

In [ ]:
plt.title('IPSI vs Indication vs label')
sns.boxplot(data=df_cvd_cleaned, x='Indication', y='IPSI', hue='label')

In [ ]:
plt.title('Contra vs Indication vs label')
sns.boxplot(data=df_cvd_cleaned, x='Indication', y='Contra', hue='label')

In [ ]:
sns.set_context('paper', font_scale=1.5)

# Set up variables in a matrix formatt.
df_ipsi_contra = df_cvd_cleaned.drop(columns=['Random', 'Id', 'Indication', 'Diabetes', 'IHD', 'Hypertension', 'Arrhythmia', 'History', 'label'])
cvd_mx = df_ipsi_contra.corr() 

sns.heatmap(cvd_mx, annot=True, cmap='RdBu')
# Slightly possitevely correlated. 

In [ ]:
# Plotting IPSI against Contra, with the label being hue.
sns.set_context('paper', font_scale=1.5)
sns.jointplot(data=df_cvd_cleaned, x='IPSI', y='Contra', kind='scatter', hue='label')
# Strong possitive correlation between Contra and IPSI leading to Risk.

# Encoding data

In [ ]:
# One-Hot Encoding Indication.
onehot_indication = pd.get_dummies(df_cvd_cleaned.Indication, prefix='Indication')
onehot_indication

In [ ]:
# The following can be dummy variables.
dummy_diabetes = pd.get_dummies(df_cvd_cleaned.Diabetes, drop_first=True)
dummy_diabetes.rename(columns={'yes' : 'Diabetes'}, inplace=True)

In [ ]:
dummy_ihd = pd.get_dummies(df_cvd_cleaned.IHD, drop_first=True)
dummy_ihd.rename(columns={'yes' : 'IHD'}, inplace=True)

In [ ]:
dummy_hypertension = pd.get_dummies(df_cvd_cleaned.Hypertension, drop_first=True)
dummy_hypertension.rename(columns={'yes' : 'Hypertension'}, inplace=True)

In [ ]:
dummy_arrhythmia = pd.get_dummies(df_cvd_cleaned.Arrhythmia, drop_first=True)
dummy_arrhythmia.rename(columns={'yes' : 'Arrhythmia'}, inplace=True)

In [ ]:
dummy_history = pd.get_dummies(df_cvd_cleaned.History, drop_first=True)
dummy_history.rename(columns={'yes' : 'History'}, inplace=True)

In [ ]:
dummy_label = pd.get_dummies(df_cvd_cleaned.label, drop_first=True)

## Feature Discretision - Chunking
Feature Discretisation helps reduce the search space. <br>
pd.cut allows us to create a new df and do some binning to the IPSI and Contra features. <br>

In [ ]:
# Define bins and labels here as both features need to be chuncked the same to keep consistency.
bins_to_chunk = [0,4,9,14,19,24,29,34,39,44,49,54,59,64,69,74,79,84,89,94,99,100]
bin_labels = ['0', '5','10','15','20','25','30','35','40','45','50','55','60','65','70','75','80','85','90','95','100']

In [ ]:
bin_ipsi = df_cvd_cleaned.IPSI
bin_ipsi = pd.cut(bin_ipsi, bins=bins_to_chunk, labels=bin_labels) # Bin Ipsilateral.
bin_ipsi.rename('IPSI_%', inplace=True) # Name the axis.
bin_ipsi = pd.to_numeric(bin_ipsi, downcast=('unsigned')) # Downcast for efficieny.
bin_ipsi.value_counts().sort_index() # Look at bins.

In [ ]:
print(bin_ipsi.isnull().sum()) # Check we havn't lost any data.

In [ ]:
# Contra already follows this patern, but bining will reduce search space.
bin_contra = df_cvd_cleaned.Contra
bin_contra = pd.cut(bin_contra, bins=bins_to_chunk, labels=bin_labels) # Bin contralaterol.
bin_contra.rename('Contra_%', inplace=True) # Name the axis.
bin_contra = pd.to_numeric(bin_contra, downcast=('unsigned')) # Downcast for efficieny.
bin_contra.value_counts().sort_index() # Look at bins.

In [ ]:
print(bin_contra.isnull().sum()) # Check we havn't lost any data.

## Amalagamate encoded features into a new df.
While getting our data ready for training, I'm also going to drop the random, and id columns as they don't hold relevant information for the model. Multiple sessions could hold a patern, they could show how early symptoms do develop into a high risk of CVD over time.

In [ ]:
# drop old columns.
df_cvd_encoded_set_0 = df_cvd_cleaned.drop(columns=['Random', 'Id', 'Indication', 'Diabetes', 'IHD', 'Hypertension', 'Arrhythmia', 'History', 'IPSI', 'Contra', 'label'])
# concat new encoded columns.
df_cvd_encoded_set_0 = pd.concat([df_cvd_encoded_set_0, onehot_indication, dummy_diabetes, dummy_ihd, dummy_hypertension, dummy_arrhythmia, dummy_history, bin_ipsi, bin_contra, dummy_label], axis=1)
df_cvd_encoded_set_0

### Now that the data is encoded, we can visualise some more patterns.

In [ ]:
plt.figure(figsize=(10,8))
plt.title('Heatmap of all features')
sns.set_context('paper', font_scale=1.1)
sns.heatmap(data=df_cvd_encoded_set_0.corr(), annot=True)

In [ ]:
sns.catplot(data=df_cvd_encoded_set_0, x='Arrhythmia', y='Contra_%', hue='Risk')
# Identified possible outlier, there is a no risk data point that has high contra and arrhythmia.

In [ ]:
# Unable to get selection to work, Contra_% is acting strange.
# df_cvd_encoded_set_0[(df_cvd_encoded_set_0.Arrhythmia == 1) & (df_cvd_encoded_set_0.Contra_% > 70)]

In [ ]:
sns.catplot(data=df_cvd_encoded_set_0, x='Arrhythmia', y='IPSI_%', hue='Risk')

## Feature Selection
I am creating 3 new data sets based on learning what features can be dropped.

In [ ]:
df_cvd_encoded_set_1 = df_cvd_encoded_set_0.drop(columns=['History'])
df_cvd_encoded_set_1.head(1)

In [ ]:
# Based off Set 1 that already has history dropped.
df_cvd_encoded_set_2 = df_cvd_encoded_set_1.drop(columns=['Indication_A-F', 'Indication_CVA'])
df_cvd_encoded_set_2.head(1)

In [ ]:
# Based off Set 1 that already has history dropped.
# Set 3 added after Random Forrest feature importance.
df_cvd_encoded_set_3 = df_cvd_encoded_set_1.drop(columns=['Indication_ASX'])
df_cvd_encoded_set_3.head(1)

## Sanity checks

First I would like to manually look at a few entries and compare the raw data to the encoded data to ensure that the encoding has been done correctly. 

In [ ]:
# Picked because of IPSI and Contra, wanted to ensure binning was correct. It initially wasn't.
df_cvd_encoded_set_0.loc[[922]] # A-F,no,no,yes,no,no,75.0,50 ,NoRisk

In [ ]:
df_cvd_encoded_set_0.dtypes # Sanity check, ensure data is all numerical and ready for model training.

In [ ]:
print(df_cvd_encoded_set_0.isnull().sum()) # Sanity check, ensure there are no nulls.

# Modelling
## Split the data

Now our data is ready, we want to create our training and testing sets. <br>

Our truth, target value y, will be Risk, as that's what we want our model to predict. <br>

Our training data, inputs x, will be everything other than Risk. <br>

We split our data 70 / 30. We train on 70% of the data and then test on 30%.



In [ ]:
# Change data sets here.
df_model_data = df_cvd_encoded_set_3

In [ ]:
# Create target set Y - Risk.
y = df_model_data.Risk
y # is series, 1d.

In [ ]:
# Create training set x - Everything BUT Risk.
x = df_model_data.drop(columns=['Risk'])
x # is data frame, 2d.

In [ ]:
# Split the data.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)

## Multi Layer Perceptron (MLP) classifier.

Now that we have our data sets split, we can pass it to our model for training. <br>

In [ ]:
# Create the MLP model.
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

# model = MLPClassifier(hidden_layer_sizes=(11,14,2), max_iter=1000, activation='relu', solver='sgd', learning_rate='adaptive') # MLP 1
# model = MLPClassifier(hidden_layer_sizes=(11,14,2), max_iter=1000, activation='identity', solver='lbfgs') # MLP 2,3,4,5
model = MLPClassifier(hidden_layer_sizes=(11,14,2), max_iter=1000, activation='tanh', solver='adam', batch_size=500, beta_1=0.8, beta_2=0.75) # MLP 6 

model.fit(x_train,y_train) # Training - Fit our data to the model.

pred_y = model.predict(x_test) # Predict. 

accuracy_score(y_test, pred_y, normalize=True)

## Evaluation metrics - Confusion matrix and F2 score

In [ ]:
mean_squared_error(y_train, model.predict(x_train))

In [ ]:
mean_absolute_error(y_train, model.predict(x_train))

In [ ]:
print(classification_report(y_test, pred_y))

In [ ]:
# We used our model to predict (x_test) above, now we are comparing that with the truth (y_test).
sns_plot = skplt.metrics.plot_confusion_matrix(y_test, pred_y, normalize=True) 
sns_plot.figure.savefig("Second model.png") # Save it as we will go through and change some things.

# Random Forest Classifier
https://medium.com/analytics-vidhya/evaluating-a-random-forest-model-9d165595ad56


In [ ]:
# Create a list to label the feature importance.
# Need to make df with labels for each of the 4 different data sets otherwise they don't match up.
df_cvd_feature_names_set_0 = ['Indication A-F', 'Indication ASX', 'Indication CVA', 'Indication TIA', 'Diabetes', 'IHD', 'Hypertension', 'Arrhythmia', 'History', 'IPSI', 'Contra']

df_cvd_feature_names_set_3 = ['Indication A-F', 'Indication CVA', 'Indication TIA', 'Diabetes', 'IHD', 'Hypertension', 'Arrhythmia', 'IPSI', 'Contra']

In [ ]:
rf = RandomForestClassifier(n_estimators=200, criterion='gini')
rf.fit( x_train, y_train ) # Use the same split data as above.
y_pred_test = rf.predict(x_test) # Predict using random forrest.
rf.score( x_test, y_test ) # View accuracy score 

In [ ]:
print(classification_report(y_test, y_pred_test)) # View the classification report for test data and predictions

In [ ]:
# Print out the feature importance, labeled.
# Only working for data set 0 atm.
for name, score in zip( df_cvd_feature_names_set_3, rf.feature_importances_ ):
    print(name, score)
# From this I can see that ASX and History have low importance.

In [ ]:
sns_plot = skplt.metrics.plot_confusion_matrix(y_test, y_pred_test, normalize=True) 